In [1]:
import h5py
import sys
import os
import glob
import csv
import pyspark
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from hdfs import InsecureClient
import numpy as np
import tables

Go through the base path and find all the .h5 files that are present there then save them to a csv file in order to load them later into spark

TODO: this actually needs to be reworked to create a table based on the hdfs file directory instead of local, however it doesn't matter as long as we don't change the files just run the code and pretend it does the right thing for now

In [2]:
basedir = '../MillionSongSubset/data'
os.listdir(basedir)

['A', 'B', 'files.csv']

In [3]:
ext='.h5'
all_files = []
for root, dirs, files in os.walk(basedir):
    #print(root, dirs, f)
    files = glob.glob(os.path.join(root,'*'+ext))
    all_files.append(files)

flat_list = [item for sublist in all_files for item in sublist]
file = '/files.csv'
with open(basedir + file, 'w', newline='') as myfile:
    for line in flat_list:
        myfile.write(line)
        myfile.write('\n')

Set up the spark application

In [4]:
# calculate the number of partitions we want for the rdds
num_nodes = 3
num_rep = 2 * num_nodes
#sc.stop()
conf = (SparkConf()
   .setMaster("spark://192.168.2.110:7077")
   .setAppName("Group14")
   .set("spark.executor.cores", 2) # if anything >2, does not run
   .set("spark.pytspark.python","python3")
   .set("spark.dynamicAllocation.enabled", False)  
   .set("spark.shuffle.service.enabled", False))

#sc = SparkContext(conf = conf)
sc = SparkContext.getOrCreate(conf)

In [5]:
sqlContext = SQLContext(sc)

code to convert filenames to actual file content

In [6]:
# good methods to explore the h5 inread: .keys(), .attrs.items() see documentation of h5py

def get_title(file, idx=0):
    return file['metadata']['songs']['title'][idx].decode("utf-8")

def get_artist_name(file, idx=0):
    return file['metadata']['songs']['artist_name'][idx].decode("utf-8")


def get_artist_familiarity(h5,songidx=0):
    """
    Get artist familiarity from a HDF5 song file, by default the first song in it
    """
    return str(h5['metadata']['songs']['artist_familiarity'][songidx]).encode('utf-8', 'ignore').decode("utf-8")

def get_artist_hotttnesss(h5,songidx=0):
    """
    Get artist hotttnesss from a HDF5 song file, by default the first song in it
    """
    return str(h5['metadata']['songs']['artist_hotttnesss'][songidx]).encode('utf-8','ignore').decode("utf-8")


def get_artist_id(h5,songidx=0):
    """
    Get artist id from a HDF5 song file, by default the first song in it
    """
    return h5['metadata']['songs']['artist_id'][songidx].decode("utf-8")

def get_artist_mbid(h5,songidx=0):
    """
    Get artist musibrainz id from a HDF5 song file, by default the first song in it
    """
    return h5['metadata']['songs']['artist_mbid'][songidx].decode("utf-8")

def get_artist_playmeid(h5,songidx=0):
    """
    Get artist playme id from a HDF5 song file, by default the first song in it
    """
    return str(h5['metadata']['songs']['artist_playmeid'][songidx]).encode('utf-8','ignore').decode("utf-8")

def get_artist_7digitalid(h5,songidx=0):
    """
    Get artist 7digital id from a HDF5 song file, by default the first song in it
    """
    return str(h5['metadata']['songs']['artist_7digitalid'][songidx]).encode('utf-8','ignore').decode("utf-8")

def get_artist_latitude(h5,songidx=0):
    """
    Get artist latitude from a HDF5 song file, by default the first song in it
    """
    return str(h5['metadata']['songs']['artist_latitude'][songidx]).encode('utf-8','ignore').decode("utf-8")

def get_artist_longitude(h5,songidx=0):
    """
    Get artist longitude from a HDF5 song file, by default the first song in it
    """
    return str(h5['metadata']['songs']['artist_longitude'][songidx]).encode('utf-8','ignore').decode("utf-8")

def get_artist_location(h5,songidx=0):
    """
    Get artist location from a HDF5 song file, by default the first song in it
    """
    return h5['metadata']['songs']['artist_location'][songidx].decode("utf-8")


def get_release(h5,songidx=0):
    """
    Get release from a HDF5 song file, by default the first song in it
    """
    return h5['metadata']['songs']['release'][songidx].decode("utf-8")

def get_release_7digitalid(h5,songidx=0):
    """
    Get release 7digital id from a HDF5 song file, by default the first song in it
    """
    return str(h5['metadata']['songs']['release_7digitalid'][songidx]).encode('utf-8','ignore').decode("utf-8")

def get_song_id(h5,songidx=0):
    """
    Get song id from a HDF5 song file, by default the first song in it
    """
    return h5['metadata']['songs']['song_id'][songidx].decode("utf-8")

def get_song_hotttnesss(h5,songidx=0):
    """
    Get song hotttnesss from a HDF5 song file, by default the first song in it
    """
    return str(h5['metadata']['songs']['song_hotttnesss'][songidx]).encode('utf-8','ignore').decode("utf-8")


def get_track_7digitalid(h5,songidx=0):
    """
    Get track 7digital id from a HDF5 song file, by default the first song in it
    """
    return str(h5['metadata']['songs']['track_7digitalid'][songidx]).encode('utf-8','ignore').decode("utf-8")


def get_analysis_sample_rate(h5,songidx=0):
    """
    Get analysis sample rate from a HDF5 song file, by default the first song in it
    """
    return str(h5['analysis']['songs']['analysis_sample_rate'][songidx]).encode('utf-8','ignore').decode("utf-8")


def get_audio_md5(h5,songidx=0):
    """
    Get audio MD5 from a HDF5 song file, by default the first song in it
    """
    return h5['analysis']['songs']['audio_md5'][songidx].decode("utf-8")


def get_danceability(h5,songidx=0):
    """
    Get danceability from a HDF5 song file, by default the first song in it
    """
    return str(h5['analysis']['songs']['danceability'][songidx]).encode('utf-8','ignore').decode("utf-8")


def get_duration(h5,songidx=0):
    """
    Get duration from a HDF5 song file, by default the first song in it
    """
    return str(h5['analysis']['songs']['duration'][songidx]).encode('utf-8','ignore').decode("utf-8")


def get_end_of_fade_in(h5,songidx=0):
    """
    Get end of fade in from a HDF5 song file, by default the first song in it
    """
    return str(h5['analysis']['songs']['end_of_fade_in'][songidx]).encode('utf-8','ignore').decode("utf-8")


def get_energy(h5,songidx=0):
    """
    Get energy from a HDF5 song file, by default the first song in it
    """
    return str(h5['analysis']['songs']['energy'][songidx]).encode('utf-8','ignore').decode("utf-8")


def get_key(h5,songidx=0):
    """
    Get key from a HDF5 song file, by default the first song in it
    """
    return str(h5['analysis']['songs']['key'][songidx]).encode('utf-8','ignore').decode("utf-8")


def get_key_confidence(h5,songidx=0):
    """
    Get key confidence from a HDF5 song file, by default the first song in it
    """
    return str(h5['analysis']['songs']['key_confidence'][songidx]).encode('utf-8','ignore').decode("utf-8")


def get_loudness(h5,songidx=0):
    """
    Get loudness from a HDF5 song file, by default the first song in it
    """
    return str(h5['analysis']['songs']['loudness'][songidx]).encode('utf-8','ignore').decode("utf-8")


def get_mode(h5,songidx=0):
    """
    Get mode from a HDF5 song file, by default the first song ifn it
    """
    return str(h5['analysis']['songs']['mode'][songidx]).encode('utf-8','ignore').decode("utf-8")


def get_mode_confidence(h5,songidx=0):
    """
    Get mode confidence from a HDF5 song file, by default the first song in it
    """
    return str(h5['analysis']['songs']['mode_confidence'][songidx]).encode('utf-8','ignore').decode("utf-8")


def get_start_of_fade_out(h5,songidx=0):
    """
    Get start of fade out from a HDF5 song file, by default the first song in it
    """
    return str(h5['analysis']['songs']['start_of_fade_out'][songidx]).encode('utf-8','ignore').decode("utf-8")


def get_tempo(h5,songidx=0):
    """
    Get tempo from a HDF5 song file, by default the first song in it
    """
    return str(h5['analysis']['songs']['tempo'][songidx]).encode('utf-8','ignore').decode("utf-8")

def get_time_signature(h5,songidx=0):
    """
    Get signature from a HDF5 song file, by default the first song in it
    """
    return str(h5['analysis']['songs']['time_signature'][songidx]).encode('utf-8','ignore').decode("utf-8")

def get_time_signature_confidence(h5,songidx=0):
    """
    Get signature confidence from a HDF5 song file, by default the first song in it
    """
    return str(h5['analysis']['songs']['time_signature_confidence'][songidx]).encode('utf-8','ignore').decode("utf-8")

def get_track_id(h5,songidx=0):
    """
    Get track id from a HDF5 song file, by default the first song in it
    """
    return str(h5['analysis']['songs']['track_id'][songidx]).encode('utf-8','ignore').decode("utf-8")

def get_year(h5,songidx=0):
    """
    Get release year from a HDF5 song file, by default the first song in it
    """
    return str(h5['musicbrainz']['songs']['year'][songidx]).encode('utf-8','ignore').decode("utf-8")

In [9]:
# df = sc.binaryFiles("hdfs://192.168.2.110:9000/LDSA/subset", minPartitions=12)
with open("/home/ubuntu/files.csv", "r", newline="\n") as f:
    results = [line.split(',')[-1][1:-2] for line in f]
results = results[0:100]

df = sc.union([sc.binaryFiles("hdfs://192.168.2.110:9000" + f) for f in results])

Py4JJavaError: An error occurred while calling o20.binaryFiles.
: java.io.IOException: No space left on device
	at sun.nio.ch.FileDispatcherImpl.write0(Native Method)
	at sun.nio.ch.FileDispatcherImpl.write(FileDispatcherImpl.java:60)
	at sun.nio.ch.IOUtil.writeFromNativeBuffer(IOUtil.java:93)
	at sun.nio.ch.IOUtil.write(IOUtil.java:65)
	at sun.nio.ch.FileChannelImpl.write(FileChannelImpl.java:211)
	at org.apache.spark.storage.CountingWritableChannel.write(DiskStore.scala:332)
	at org.apache.spark.util.io.ChunkedByteBuffer$$anonfun$writeFully$1.apply(ChunkedByteBuffer.scala:81)
	at org.apache.spark.util.io.ChunkedByteBuffer$$anonfun$writeFully$1.apply(ChunkedByteBuffer.scala:69)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:186)
	at org.apache.spark.util.io.ChunkedByteBuffer.writeFully(ChunkedByteBuffer.scala:69)
	at org.apache.spark.storage.DiskStore$$anonfun$putBytes$1.apply(Disk

In [33]:
df.map(lambda x: read(x[1])).take(10)

[['Casual',
  "I Didn't Mean To",
  '0.5817937658450281',
  '0.4019975433642836',
  'ARD7TVE1187B99BFB1',
  'e77e51a5-4761-45b3-9847-2051f811e366',
  '4479',
  '165270',
  'nan',
  'nan',
  'California - LA',
  'Fear Itself',
  '300848',
  'SOMZWCG12A8C13C480',
  '0.6021199899057548',
  '3401791',
  '22050',
  'a222795e07cd65b7a530f1346f520649',
  '0.0',
  '218.93179',
  '0.247',
  '0.0',
  '1',
  '0.736',
  '-11.197',
  '0',
  '0.636',
  '218.932',
  '92.198',
  '4',
  '0.778',
  "b'TRAAAAW128F429D538'",
  '0'],
 ['The Box Tops',
  'Soul Deep',
  '0.6306300375898077',
  '0.4174996449709784',
  'ARMJAGH1187FB546F3',
  '1c78ab62-db33-4433-8d0b-7c8dcf1849c2',
  '22066',
  '1998',
  '35.14968',
  '-90.04892',
  'Memphis, TN',
  'Dimensions',
  '300822',
  'SOCIWDW12A8C13D406',
  'nan',
  '3400270',
  '22050',
  'bb9771eeef3d5b204a3c55e690f52a91',
  '0.0',
  '148.03546',
  '0.148',
  '0.0',
  '6',
  '0.169',
  '-9.843',
  '0',
  '0.43',
  '137.915',
  '121.274',
  '4',
  '0.384',
  "b'TRAA

In [22]:
def read(x):
    with h5py.File(io.BytesIO(x), 'r') as f:
        song = []
        
        song.append(get_artist_name(f))
        song.append(get_title(f))
        song.append(get_artist_familiarity(f))
        song.append(get_artist_hotttnesss(f))
        song.append(get_artist_id(f))
        song.append(get_artist_mbid(f))
        song.append(get_artist_playmeid(f))
        song.append(get_artist_7digitalid(f))
        song.append(get_artist_latitude(f))
        song.append(get_artist_longitude(f))
        song.append(get_artist_location(f))
        song.append(get_release(f))
        song.append(get_release_7digitalid(f))
        song.append(get_song_id(f))
        song.append(get_song_hotttnesss(f))
        song.append(get_track_7digitalid(f))
        song.append(get_analysis_sample_rate(f))
        song.append(get_audio_md5(f))
        song.append(get_danceability(f))
        song.append(get_duration(f))
        song.append(get_end_of_fade_in(f))
        song.append(get_energy(f))
        song.append(get_key(f))
        song.append(get_key_confidence(f))
        song.append(get_loudness(f))
        song.append(get_mode(f))
        song.append(get_mode_confidence(f))
        song.append(get_start_of_fade_out(f))
        song.append(get_tempo(f))
        song.append(get_time_signature(f))
        song.append(get_time_signature_confidence(f))
        song.append(get_track_id(f))
        song.append(get_year(f))
        
    return song

In [17]:
# todo: set the number of partitions here corectly as calculated above in order to maintain speed

fp = "hdfs://192.168.2.110:9000/LDSA/subset/files.csv"
#print(fp)


def binary(x):
    return sc.binaryFiles("hdfs://192.168.2.110:9000/LDSA/subset" + x[25:])

def h5file(x):
    return h5py.File(io.BytesIO(x[0][1]), 'r')

def trial(x):
    with h5py.File("hdfs://192.168.2.110:9000/LDSA/subset" +x[25:]) as f:
        result = f[x[32:]]
        return list(result[:])
    

file_paths = sc.textFile(fp, minPartitions=12)
print(file_paths.take(1)[0][25:])
#flatMap(lambda x: binary(x))
#rdd = sc.union([sc.binaryFiles(f) for f in file_paths])

#rdd = file_paths.flatMap(binary)
#rdd.take(1)
#files.take(1)
df = sc.binaryFiles("hdfs://192.168.2.110:9000/LDSA/subset", minPartitions=12) # returns a JavaPairRDD
#ff = sqlContext.read.binaryFile("hdfs://192.168.2.110:9000/LDSA/subset")   
#print(df.take(1))
#df2=df.flatMap(h5file)
#df2 = df.map(lambda x: h5file(x))
#print(df2.take(1))
print(df)


files = df.map(lambda x: foo(x[1]))  

def foo(data):  
    with io.BytesIO(data) as b, h5py.File(b, 'r') as f:  
        return list(f) # ["analysis", "metadata", "musicbrainz"]  

#print(files.collect())


## Old version
#file_paths = sc.textFile(fp, minPartitions=12).map(lambda x: f2(x))
#print(file_paths)


## One file test
#file_paths = sc.binaryFiles("hdfs://192.168.2.110:9000/LDSA/subset/A/E/E/TRAEELO128F425BD8F.h5")
#a = h5py.File(io.BytesIO(file_paths.take(1)[0][1]), 'r')
#print(a)

group,284446,2020-06-02,09:57,"/LDSA/subset/A/A/A/TRAAAAW128F429D538.h5"
org.apache.spark.api.java.JavaPairRDD@6d2c54b3


In [19]:
print(file_paths.take(1))

['../MillionSongSubset/data/A/E/E/TRAEELO128F425BD8F.h5']


In [20]:
files = open(basedir + "/files.csv", "r")
rdd = sc.union([sc.binaryFiles("hdfs://192.168.2.110:9000/LDSA/subset" + f.rstrip("n")[25:]) for f in files])


Py4JJavaError: An error occurred while calling o20.binaryFiles.
: java.lang.OutOfMemoryError: Java heap space
	at java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:57)
	at java.nio.ByteBuffer.allocate(ByteBuffer.java:335)
	at org.apache.spark.broadcast.TorrentBroadcast$$anonfun$3.apply(TorrentBroadcast.scala:286)
	at org.apache.spark.broadcast.TorrentBroadcast$$anonfun$3.apply(TorrentBroadcast.scala:286)
	at org.apache.spark.util.io.ChunkedByteBufferOutputStream.allocateNewChunkIfNeeded(ChunkedByteBufferOutputStream.scala:87)
	at org.apache.spark.util.io.ChunkedByteBufferOutputStream.write(ChunkedByteBufferOutputStream.scala:75)
	at net.jpountz.lz4.LZ4BlockOutputStream.flushBufferedData(LZ4BlockOutputStream.java:220)
	at net.jpountz.lz4.LZ4BlockOutputStream.write(LZ4BlockOutputStream.java:173)
	at java.io.ObjectOutputStream$BlockDataOutputStream.drain(ObjectOutputStream.java:1877)
	at java.io.ObjectOutputStream$BlockDataOutputStream.write(ObjectOutputStream.java:1848)
	at java.io.Obje

In [ ]:
csv_file =open('../MillionSongSubset/data/files.csv', "r", newline="")

hdfs_file_paths = ["hdfs://192.168.2.110:9000/LDSA/subset" + path[25:].rstrip('\n') for path in csv_file]

print(hdfs_file_paths[:10])
files = sc.union([sc.binaryFiles(p) for p in hdfs_file_paths]) #.map(lambda x: foo(x[1]))
files.take(1)
def foo(data):
    with io.BytesIO(data) as b, h5py.File(b, 'r') as f:  
        return list(f) # ["analysis", "metadata", "musicbrainz"] 

# Step 3: Convert rdd to dataframe
convert the rdd (api version 1) to a dataframe (api version 2)

In [ ]:
columns = ['artist_name', 'title', 'artist_familiarity', 
             'artist_hotttnesss', 'artist_id', 'artist_mbid', 
             'artist_playmeid', 'artist_7digitalid', 'artist_latitude', 
             'artist_longitude', 'artist_location', 'release', 
             'release_7digitalid', 
             'song_id', 'song_hotnesss', 'track_7digitalid', 
             'analysis_sample_rate', 
             'audio_md5', 'danceability', 'duration', 'end_of_fade_in', 
             'energy', 'key', 
             'key_confidence', 'loudness', 'mode', 'mode_confidence', 
             'start_of_fade_out', 
             'tempo', 'time_signature', 
             'time_signature_confidence', 'track_id', 'year']
df = file_paths.toDF(columns)
df.printSchema()

In [ ]:
# change types
from pyspark.sql import types 
['BinaryType', 'BooleanType', 'ByteType', 'DateType', 
          'DecimalType', 'DoubleType', 'FloatType', 'IntegerType', 
           'LongType', 'ShortType', 'StringType', 'TimestampType']
changedTypedf = df.withColumn("year", df["year"].cast("Integer"))\
                .withColumn("track_id", df["track_id"].cast("Integer"))\
                .withColumn("artist_id", df["artist_id"].cast("Integer"))\
                .withColumn("song_id", df["song_id"].cast("Integer"))\
                .withColumn("duration", df["duration"].cast("Float"))\
                .withColumn("danceability", df["danceability"].cast("Float"))\
                .withColumn("end_of_fade_in", df["end_of_fade_in"].cast("Float"))\
                .withColumn("energy", df["energy"].cast("Float"))\
                .withColumn("key_confidence", df["key_confidence"].cast("Float"))\
                .withColumn("mode_confidence", df["mode_confidence"].cast("Float"))\
                .withColumn("release_7digitalid", df["release_7digitalid"].cast("Integer"))\
                .withColumn("song_hotnesss", df["song_hotnesss"].cast("Float"))\
                .withColumn("start_of_fade_out", df["start_of_fade_out"].cast("Float"))\
                .withColumn("loudness", df["loudness"].cast("Float"))\
                .withColumn("tempo", df["tempo"].cast("Float"))

start with the data processing

In [ ]:
changedTypedf.printSchema()

start with some very basic metrics over the dataset:

In [ ]:
changedTypedf.take(1)

count the number of different artists in the dataset and count the numbers of songs release every year

In [ ]:
changedTypedf.groupBy('artist_name')\
             .count()\
             .show()

In [ ]:
changedTypedf.groupBy("year")\
             .count()\
             .show()

Let's get a bit more advanced: Calculate the average loudness of the songs of an artists tat puplished a song after 2000

In [ ]:
from pyspark.sql.functions import col, avg
from pyspark.sql import functions as F
from pyspark.sql.functions import *

left = changedTypedf.select("artist_name").distinct().filter(changedTypedf['year'] > 2000)

right = changedTypedf.groupBy("artist_name")\
            .agg(avg(col("loudness"))\
            .alias("avg_loudness"))\
            .orderBy("avg_loudness", ascending=False)

In [ ]:
avg_loudness = left.join(right, left.artist_name == right.artist_name)\
            .select(right["artist_name"], "avg_loudness")\
            .orderBy("avg_loudness", ascending=False)\
            .collect()


In [ ]:
sc.stop()
